# Original

In [1]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from onlineTripletloss import *
from selector import *
# from model_copy import NetG, NetD, NetA, NetP
from model_SAGAN1_1 import NetG, NetD, NetA
# from model_WGANGP import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_siGAN import NetG, NetD, NetA
# from data_set import CASIABDataset
from dataset2Loader import CASIABDataset
import torch.optim as optim
import visdom
from torchvision.utils import make_grid
# Data_Dir = '../GaitRecognition/DatasetB_GEI_64x64_allseq/'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_64x64_SAGAN_90_trial11'
Model_dir = './Transform_Model/'+ Model_Name
if not os.path.isdir(Model_dir):
    os.mkdir(Model_dir)

In [ ]:

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
import os

for r, d, files in os.walk(Data_Dir):
    print(r)
    print(len(d))
    print(len(files))
    
# cpt = sum([len(files) for r, d, files in os.walk(Data_Dir)])
# cpt = sum([len(d) for r, d, files in os.walk(Data_Dir)])
# print(cpt)
# list = os.listdir(Data_Dir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# import fnmatch
# print(len(fnmatch.filter(os.listdir(Data_Dir), '*.png')))

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(16)

## iteration

In [1]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.xavier_normal_(tensor, gain=1.)
#         init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print('Training starts')
while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
    label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, label)
    lossD += lossD_real1.item()
    lossD_real1.backward()

    label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, label)
    lossD += lossD_real2.item()
    lossD_real2.backward()

    fake = netg(img).detach()
    label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, label)
    lossD += lossD_fake.item()
    lossD_fake.backward()

    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

    label.fill_(real_label)
    output1 = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output1, label)
    lossA += lossA_real1.item()
    lossA_real1.backward()

    label.fill_(fake_label)
    output = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output, label)
    lossA += lossA_real2.item()
    lossA_real2.backward()

    label.fill_(fake_label)
    output = neta(faked)
    lossA_fake = F.binary_cross_entropy(output, label)
    lossA += lossA_fake.item()
    lossA_fake.backward()
    
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)

    label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, label)
    lossG += lossGD.item()
    lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
    label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, label)
    lossG += lossGA.item()
    lossGA.backward()
    
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 25
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 1000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

    if iteration % 100==0 or iteration==10 :
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')
            
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))


SyntaxError: invalid syntax (<ipython-input-1-d2306f43491b>, line 129)

## Epoch

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
train_loader = th.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=False)
train_loader.next()

In [2]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        



n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 3.3411221504211426, ErrA = 0.44111086055636406, ErrD = 0.036158584679166474
Epoch = 4, ErrG = 3.361356258392334, ErrA = 0.3775310466686885, ErrD = 0.036184978360931076
Epoch = 6, ErrG = 4.272621393203735, ErrA = 0.3271034111579259, ErrD = 0.003417431997756163
Epoch = 8, ErrG = 4.160229206085205, ErrA = 0.2504531896362702, ErrD = 0.05292314477264881
Epoch = 10, ErrG = 3.3400180339813232, ErrA = 0.2544544863825043, ErrD = 0.3592479278643926
Epoch = 12, ErrG = 2.230285406112671, ErrA = 0.0737597147623698, ErrD = 0.4638359546661377
Epoch = 14, ErrG = 3.333204686641693, ErrA = 0.15362303347016373, ErrD = 0.3380258431037267
Epoch = 16, ErrG = 3.0305097103118896, ErrA = 0.12443562845389049, ErrD = 0.18306180338064829
Epoch = 18, ErrG = 3.0286770462989807, ErrA = 0.11020504403859377, ErrD = 0.4389033814271291
Epoch = 20, ErrG = 2.9920820891857147, ErrA = 0.020810067964096863, ErrD = 0.3843097488085429
E

KeyboardInterrupt: 

## update k times

In [2]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_img), 1)
            noassd = th.cat((img, noass_img), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))    
        

n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 1.5588033199310303, ErrA = 0.4954340805610021, ErrD = 0.19129109879334769
Epoch = 4, ErrG = 1.8620306849479675, ErrA = 0.38821983337402344, ErrD = 0.15181769927342734
Epoch = 6, ErrG = 2.9678717851638794, ErrA = 0.39058470105131465, ErrD = 0.07360446825623512
Epoch = 8, ErrG = 3.080983877182007, ErrA = 0.37689451904346544, ErrD = 0.10360063364108403
Epoch = 10, ErrG = 2.6229763627052307, ErrA = 0.35657545427481335, ErrD = 0.4389157295227051
Epoch = 12, ErrG = 2.3641445338726044, ErrA = 0.3480776399374008, ErrD = 0.4964556197325389
Epoch = 14, ErrG = 2.986826181411743, ErrA = 0.19838610012084246, ErrD = 0.6122767329216003
Epoch = 16, ErrG = 3.181006968021393, ErrA = 0.6149165614818534, ErrD = 0.5359568235774835
Epoch = 18, ErrG = 3.4160876870155334, ErrA = 0.629215250412623, ErrD = 0.030627019703388214
Epoch = 20, ErrG = 3.121281534433365, ErrA = 0.3222902840934694, ErrD = 0.9301179846127828
Epoc

Epoch = 176, ErrG = 6.831139832735062, ErrA = 0.004716723418823676, ErrD = 0.3077218135197957
Epoch = 178, ErrG = 5.7401123046875, ErrA = 0.0005244686411363849, ErrD = 0.19037060687939325
Epoch = 180, ErrG = 5.42939567565918, ErrA = 0.0009260319119069512, ErrD = 0.32404774862031144
Epoch = 182, ErrG = 4.795043647289276, ErrA = 0.0009413289275149358, ErrD = 0.5196272283792496
Epoch = 184, ErrG = 5.475306749343872, ErrA = 0.00041795648091162246, ErrD = 0.2979186934729417
Epoch = 186, ErrG = 6.479019582271576, ErrA = 0.0007185266852805702, ErrD = 0.17575795451800028
Epoch = 188, ErrG = 6.157017111778259, ErrA = 0.00047458595751474303, ErrD = 0.1468967745701472
Epoch = 190, ErrG = 5.370856821537018, ErrA = 0.025097796693444252, ErrD = 0.20363212128480276
Epoch = 192, ErrG = 5.994137287139893, ErrA = 0.0019459321180571958, ErrD = 0.1704980581998825
Epoch = 194, ErrG = 6.887574553489685, ErrA = 0.0005688234808379397, ErrD = 0.2908540566762288
Epoch = 196, ErrG = 5.431972861289978, ErrA = 0.0

Epoch = 348, ErrG = 6.1596128940582275, ErrA = 0.00013765553497554114, ErrD = 0.11682136698315541
Epoch = 350, ErrG = 9.763295352458954, ErrA = 3.4831571914158608e-06, ErrD = 0.161186791335543
Epoch = 352, ErrG = 8.154533386230469, ErrA = 0.00024326466035518024, ErrD = 0.2682118167479833
Epoch = 354, ErrG = 7.361542344093323, ErrA = 0.0002493309823421441, ErrD = 0.1081374188264211
Epoch = 356, ErrG = 5.796618342399597, ErrA = 0.000159685312610236, ErrD = 0.21990248436729112
Epoch = 358, ErrG = 7.202744960784912, ErrA = 2.3819473578138666e-05, ErrD = 0.1321813693890969
Epoch = 360, ErrG = 6.564028263092041, ErrA = 9.72373321322569e-05, ErrD = 0.11159980359176795
Epoch = 362, ErrG = 6.711366355419159, ErrA = 0.0002984184033039128, ErrD = 0.11192851513624191
Epoch = 364, ErrG = 7.656510949134827, ErrA = 3.456297205654361e-05, ErrD = 0.11052152638634045
Epoch = 366, ErrG = 8.382885754108429, ErrA = 0.00014327441052349363, ErrD = 0.11537620301047961
Epoch = 368, ErrG = 7.920348703861237, Er

Epoch = 520, ErrG = 7.254403591156006, ErrA = 0.00013418334615285707, ErrD = 0.08429891072834532
Epoch = 522, ErrG = 7.825333952903748, ErrA = 7.228213871712796e-05, ErrD = 0.21718123431007066
Epoch = 524, ErrG = 8.774211645126343, ErrA = 3.339733822826929e-06, ErrD = 0.13289840271075568
Epoch = 526, ErrG = 7.989862322807312, ErrA = 5.2516104346977954e-05, ErrD = 0.16965349266926447
Epoch = 528, ErrG = 7.672500789165497, ErrA = 0.00016228325699785273, ErrD = 0.11582544383903344
Epoch = 530, ErrG = 7.485991954803467, ErrA = 0.0005245358292995661, ErrD = 0.22601295510927835
Epoch = 532, ErrG = 8.76667594909668, ErrA = 2.5884011696083082e-05, ErrD = 0.20086409151554108
Epoch = 534, ErrG = 6.727994441986084, ErrA = 1.877761421080019e-05, ErrD = 0.3139570926626523
Epoch = 536, ErrG = 8.76380729675293, ErrA = 4.5561538475643225e-05, ErrD = 0.08143636522193749
Epoch = 538, ErrG = 7.52375340461731, ErrA = 0.00285392347724193, ErrD = 0.1418602019548416
Epoch = 540, ErrG = 8.153590440750122, Err

Epoch = 692, ErrG = 6.815811634063721, ErrA = 0.00046585347100365954, ErrD = 0.05685470253229141
Epoch = 694, ErrG = 8.090083122253418, ErrA = 0.0001284888943284083, ErrD = 0.11047945295770963
Epoch = 696, ErrG = 8.739537358283997, ErrA = 1.194665545275105e-05, ErrD = 0.18067911826074123
Epoch = 698, ErrG = 7.190846085548401, ErrA = 6.725269486670034e-05, ErrD = 0.17465874056021372
Epoch = 700, ErrG = 7.692009449005127, ErrA = 6.864315734371e-05, ErrD = 0.08567465655505657


## model changed +Dp

In [2]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netp = NetP(nc=1)
neta = NetA(nc=1)
netd = NetD(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netp.children())[0].children(),
    list(neta.children())[0].children(),
    list(netd.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netp = netp.to(device)
neta = neta.to(device)
netd = netd.to(device)
netg.train()
netp.train()
neta.train()
netd.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimP = optim.Adam(netp.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        
        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()


        # update P
        lossP = 0
        optimP.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossP_real1 = F.binary_cross_entropy(output1, label)
        lossP += lossP_real1.item()/2
        lossP_real1 = lossP_real1/2
        lossP_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossP_real2 = F.binary_cross_entropy(output, label)
        lossP += lossP_real2.item()/2
        lossP_real2 = lossP_real2/2
        lossP_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossP_fake = F.binary_cross_entropy(output, label)
        lossP += lossP_fake.item()/2
        lossP_fake = lossP_fake/2
        lossP_fake.backward()

        optimP.step()
    

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        
        fake = netg(img)
        output = netd(fake)
        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward(retain_graph=True)
        
        faked = th.cat((img, fake), 1)
        output = netp(faked)
        label.fill_(real_label)
        lossGP = F.binary_cross_entropy(output, label)
        lossG += lossGP.item()/2
        lossGP = lossGP/2
        lossGP.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch, epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD', 'lossP']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch, epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}  \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        



n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 3.11608849465847, ErrA = 0.3182279219230016, ErrD = 0.047072598710656166, ErrP = 0.635548980285724
Epoch = 4, ErrG = 4.2982336431741714, ErrA = 0.2356738063196341, ErrD = 0.01883645790318648, ErrP = 0.9297461515913407
Epoch = 6, ErrG = 1.0726338624954224, ErrA = 0.4498101423184077, ErrD = 0.5129949947198232, ErrP = 0.795226939022541
Epoch = 8, ErrG = 1.4125598818063736, ErrA = 0.40818140904108685, ErrD = 0.35858893394470215, ErrP = 0.7477960462371508
Epoch = 10, ErrG = 1.2971711158752441, ErrA = 0.3943396210670471, ErrD = 0.5168332060178121, ErrP = 0.7842573672533035
Epoch = 12, ErrG = 1.287300482392311, ErrA = 0.3967048724492391, ErrD = 0.4421286384264628, ErrP = 0.8709859227140745
Epoch = 14, ErrG = 1.0789260566234589, ErrA = 0.45353100697199505, ErrD = 0.5167980094750723, ErrP = 0.774136196821928
Epoch = 16, ErrG = 0.8869864493608475, ErrA = 0.4371970593929291, ErrD = 0.6796552538871765, ErrP

Epoch = 138, ErrG = 1.8848372548818588, ErrA = 0.37690639923675917, ErrD = 0.126130573451519, ErrP = 1.3815680195887883
Epoch = 140, ErrG = 2.459674745798111, ErrA = 0.13985291744271913, ErrD = 0.052301390717426934, ErrP = 1.046545887986819
Epoch = 142, ErrG = 2.4358335584402084, ErrA = 0.34625486967464286, ErrD = 0.04889713848630587, ErrP = 1.2343868588407834
Epoch = 144, ErrG = 3.460209161043167, ErrA = 0.17166305863065645, ErrD = 0.06769575364887714, ErrP = 1.2942327298223972
Epoch = 146, ErrG = 2.29889839887619, ErrA = 0.24675471087296805, ErrD = 0.06533551961183548, ErrP = 1.054988423983256
Epoch = 148, ErrG = 2.555293306708336, ErrA = 0.19735413370653987, ErrD = 0.06869085381428401, ErrP = 1.0796024451653163
Epoch = 150, ErrG = 2.8094538152217865, ErrA = 0.0653710098316272, ErrD = 0.09579195268452168, ErrP = 1.0603360533714294
Epoch = 152, ErrG = 2.5584416687488556, ErrA = 0.15072270280991992, ErrD = 0.05339148143927256, ErrP = 1.2721886957685153
Epoch = 154, ErrG = 3.06523436307

Epoch = 274, ErrG = 4.246414482593536, ErrA = 0.07158860199463864, ErrD = 0.017607325144732993, ErrP = 1.0888368937497337
Epoch = 276, ErrG = 5.505197733640671, ErrA = 0.01899628232543667, ErrD = 0.036937227783103786, ErrP = 1.6035501429190238
Epoch = 278, ErrG = 5.23977318406105, ErrA = 0.007394160100375302, ErrD = 0.005038390149517606, ErrP = 1.736711440763126
Epoch = 280, ErrG = 3.1377120912075043, ErrA = 0.08311231819849733, ErrD = 0.014951018461336693, ErrP = 1.611320825914542
Epoch = 282, ErrG = 1.8737804591655731, ErrA = 0.10378054933001597, ErrD = 0.14773708581924438, ErrP = 1.1504630371928215
Epoch = 284, ErrG = 4.510046571493149, ErrA = 0.01151328650303185, ErrD = 0.01227961697926124, ErrP = 1.310951016843319
Epoch = 286, ErrG = 2.898777276277542, ErrA = 0.1016156747743177, ErrD = 0.021115196558336418, ErrP = 1.3343367502093315
Epoch = 288, ErrG = 3.0274031162261963, ErrA = 0.08210172741140316, ErrD = 0.016999583691358566, ErrP = 1.3390904019276302
Epoch = 290, ErrG = 3.39559

Epoch = 410, ErrG = 3.533279463648796, ErrA = 0.18423704275240502, ErrD = 0.02176489882792036, ErrP = 1.0999428828557332
Epoch = 412, ErrG = 4.836178123950958, ErrA = 0.025415120626954984, ErrD = 0.014120173950990042, ErrP = 1.5275077670812607
Epoch = 414, ErrG = 4.70133513212204, ErrA = 0.02017675046226941, ErrD = 0.018193533644080162, ErrP = 1.4544565848385294
Epoch = 416, ErrG = 3.393650382757187, ErrA = 0.09150503514668647, ErrD = 0.016030723229050636, ErrP = 1.7136773901681106
Epoch = 418, ErrG = 5.0546484142541885, ErrA = 0.026691945117969833, ErrD = 0.0371281080879271, ErrP = 1.7391536757349968
Epoch = 420, ErrG = 4.21792396903038, ErrA = 0.008521542561841974, ErrD = 0.0178439828256766, ErrP = 1.5578158454348643
Epoch = 422, ErrG = 4.1009859293699265, ErrA = 0.037448210399209834, ErrD = 0.02382975568373998, ErrP = 1.2971549574285746
Epoch = 424, ErrG = 5.2505334466695786, ErrA = 0.06564981438956845, ErrD = 0.020380170550197363, ErrP = 1.4769948869943619
Epoch = 426, ErrG = 4.590

Epoch = 546, ErrG = 5.477801129221916, ErrA = 0.002713428737782427, ErrD = 0.019839648467799027, ErrP = 2.1226077305230624
Epoch = 548, ErrG = 4.065415054559708, ErrA = 0.02102125537445924, ErrD = 0.01617185352370143, ErrP = 1.604040424649914
Epoch = 550, ErrG = 4.537599295377731, ErrA = 0.02919076700466879, ErrD = 0.018528030486777425, ErrP = 1.8433641091299553
Epoch = 552, ErrG = 5.152814447879791, ErrA = 0.00454806725247181, ErrD = 0.006922723812749609, ErrP = 1.7361562778241932
Epoch = 554, ErrG = 5.766420602798462, ErrA = 0.002164118593403449, ErrD = 0.003644603304564953, ErrP = 1.7609650113154203
Epoch = 556, ErrG = 6.486919894814491, ErrA = 0.015705314464867115, ErrD = 0.015900192471841972, ErrP = 1.5003225519321859
Epoch = 558, ErrG = 3.902671664953232, ErrA = 0.029854742927303352, ErrD = 0.009534214972518384, ErrP = 1.8170404036839802
Epoch = 560, ErrG = 5.419224604964256, ErrA = 0.011814371981017757, ErrD = 0.007371207776789864, ErrP = 1.7574417429665725
Epoch = 562, ErrG = 4

Epoch = 682, ErrG = 4.614795744419098, ErrA = 0.003497906973128314, ErrD = 0.043314843981837235, ErrP = 2.3817883241960467
Epoch = 684, ErrG = 6.708198115229607, ErrA = 0.0009738096089980294, ErrD = 0.02778815571218729, ErrP = 2.4185923703577523
Epoch = 686, ErrG = 3.5164002031087875, ErrA = 0.07445311649159218, ErrD = 0.011461217266817888, ErrP = 1.6208183467388153
Epoch = 688, ErrG = 6.071031093597412, ErrA = 0.0023168805685950624, ErrD = 0.016358414975305397, ErrP = 2.3647002772195265
Epoch = 690, ErrG = 5.7160618007183075, ErrA = 0.004689266908068627, ErrD = 0.007368836629514893, ErrP = 2.111980510642752
Epoch = 692, ErrG = 2.6602130830287933, ErrA = 0.13904298325845352, ErrD = 0.07406613665322463, ErrP = 1.7235653350750606
Epoch = 694, ErrG = 4.651394456624985, ErrA = 0.006315744171691525, ErrD = 0.014701074299712976, ErrP = 2.127034991669158
Epoch = 696, ErrG = 5.875986486673355, ErrA = 0.006327504727475268, ErrD = 0.006433010722200076, ErrP = 2.0806407686322927
Epoch = 698, ErrG

# Triplet 

In [ ]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from model_siGAN import NetG, NetD, NetA
from data_set import *
import torch.optim as optim
from torchvision.utils import make_grid

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(1)
# ass_label, noass_label, img = dataset.getbatch(1)

# dataset_test = CASIABDatasetForTest(data_dir=Data_Dir)
# ass_label_test, noass_label_test, img_test = dataset_test.getbatch(1)
# ass_label_test, noass_label_test, img_test = dataset_test.getbatch(1)

In [ ]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from model_siGAN import NetG, NetD, NetA
from data_set import *
import torch.optim as optim
from torchvision.utils import make_grid
import visdom
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_Triplet'
if not os.path.isdir(Model_Name):
    os.mkdir(Model_Name)
    
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
#     list(netd.children())[0].children(),
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netg.train()
dataset = CASIABDataset(data_dir=Data_Dir)
dataset_test = CASIABDatasetForTest(data_dir=Data_Dir)
ass_label_test, noass_label_test, img_test = dataset_test.getbatch(300)
ass_label_test = ass_label_test.to(device).to(th.float32)
noass_label_test = noass_label_test.to(device).to(th.float32)
img_test = img_test.to(device).to(th.float32)

iteration = 0
lr = 0.0001
batchSize = 128

optimG = optim.Adam(netg.parameters(), lr=lr)
# optimG = optim.Adam(netg.parameters(), lr=lr, weight_decay=1e-4)

while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    __, A = netg(img)
    __, P = netg(ass_label)
    __, N = netg(noass_label)
    lossTriplet = F.triplet_margin_loss(A, P, N, margin=5)
    lossGT = lossTriplet.item()
    lossTriplet.backward()
    optimG.step()
    
    iteration+=1
    if iteration % 500 == 0:
        with th.no_grad():
            netg.eval()  #切換
            __, A_test = netg(img_test)
            __, P_test = netg(ass_label_test)
            __, N_test = netg(noass_label_test)
            loss_test = F.triplet_margin_loss(A_test, P_test, N_test, margin=5)
            loss_test = loss_test.item()
            netg.train() #切換回去
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrGT = {}, Test_ErrGT = {}\n'.format(
            iteration, lossGT, loss_test
        ))
        print('iter = {}, ErrGT = {}, Test_ErrGT = {}'.format(
            iteration, lossGT, loss_test
        ))

        if win is None:
            win = vis.line(X=np.array([[iteration,iteration]]),
                           Y=np.array([[lossGT, loss_test]]),
                           opts=dict(
                               title= Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossGT', 'lossGT_test']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration]]),
                     Y=np.array([[lossGT, loss_test]]),
                     win=win,
                     update='append')
            
    if iteration % 2500 == 0:
        state = {
#             'netA': neta.state_dict(),
            'netG': netg.state_dict(),
#             'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

# Combine Siamese and GAN 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
margin = 5
target = '036'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, margin = {}, target={} \n'.format(
            epoches, lr, batchSize, margin, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake, _ = netg(img)
        label.fill_(fake_label)
        output2 = netd(fake.detach())
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()

        # update A
        lossA = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake,_ = netg(img)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked.detach())
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()
        
        
        # update G
        lossG = 0
        optimG.zero_grad()
        fake, A = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()
        
        # constrain on generator
        fake_ass, P = netg(ass_label)
        fake_noass, N = netg(noass_label)
        lossTriplet = F.triplet_margin_loss(fake, fake_ass, fake_noass, margin = margin)
        lossG += lossTriplet.item()
        lossTriplet.backward()
        
        # constrain on encoder
#         __, P = netg(ass_label)
#         __, N = netg(noass_label)
#         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
#         lossG += lossTriplet.item()
#         lossTriplet.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# Combine Siamese and GAN k times 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
margin = 10
g_k = 2
k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, margin = {}, g_k ={}, target={} \n'.format(
            epoches, lr, batchSize, margin, g_k, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
        
#         if(i ==0):
#             print(label_neg,label_anc,label_pos)
#             print(com_label)
#         print("shape",ass_label.shape,noass_label.shape,img.shape,com_img.shape, com_label.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach()) #需要 detach 因為不希望更新fake的參數
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#     if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake, A = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)  ##這裡需要retain graph 因為他之後有需要fake，因此需要retain

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()
        
        ## new tripletloss
        _, P = netg(ass_img)
        __, N = netg(noass_img)
        lossf = TripletLoss(margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
        lossTriplet =lossf(A, P, N)
        lossG += lossTriplet.item()
        lossTriplet.backward()
        
#         ## new onlinetripletloss
#         __, com = netg(com_img)
#         loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
#         lossTriplet,len_triplet = loss_fn(com, com_label)
#         lossG += lossTriplet.item()
#         lossTriplet.backward()
# #         print(lossTriplet.item(),len_triplet)
        
#         ## triplet loss
#         __, P = netg(ass_img)
#         __, N = netg(noass_img)
#         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
#         lossG += lossTriplet.item()
#         lossTriplet.backward()
# #         if i%10==0:
# #             print("tripletloss ",lossTriplet.item())

        ## tripletloss no negative
#         N_plus = th.zeros((A.size()), requires_grad=False).to(device)
#         lossTriplet_AP = F.triplet_margin_loss(A, P, N_plus, margin = margin)
#         lossG += lossTriplet_AP.item()
#         lossTriplet += lossTriplet_AP
#         lossTriplet.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG  Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 6.079311847686768, ErrA = 0.45719657093286514, ErrD = 0.15815530717372894
Epoch = 4, ErrG = 4.7586734890937805, ErrA = 0.42604386806488037, ErrD = 0.42887693643569946
Epoch = 6, ErrG = 3.702850639820099, ErrA = 0.47924991448720294, ErrD = 0.494888573884964
Epoch = 8, ErrG = 3.5415768027305603, ErrA = 0.5032813847064972, ErrD = 0.6257400711377462
Epoch = 10, ErrG = 3.145357310771942, ErrA = 0.569260984659195, ErrD = 0.5315638681252798
Epoch = 12, ErrG = 2.8632413148880005, ErrA = 0.5580846269925436, ErrD = 0.6164103746414185
Epoch = 14, ErrG = 3.4398266077041626, ErrA = 0.5699803531169891, ErrD = 0.5714153349399567
Epoch = 16, ErrG = 2.488332360982895, ErrA = 0.5693407853444418, ErrD = 0.5842624207337698
Epoch = 18, ErrG = 2.8794360756874084, ErrA = 0.5267928838729858, ErrD = 0.6221688290437063
Epoch = 20, Err

# from PixelDT code

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

real_label = th.zeros((batchSize, 1), requires_grad=False).to(device)
fake_label = th.ones((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr)
optimD = optim.Adam(netd.parameters(), lr=lr)
optimA = optim.Adam(neta.parameters(), lr=lr)

print('Training starts')
while iteration < 40000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
#     label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, real_label)
#     lossD += lossD_real1.item()
#     lossD_real1.backward()

#     label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, real_label)
#     lossD += lossD_real2.item()
#     lossD_real2.backward()

    fake = netg(img).detach()
#     label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, fake_label)
#     lossD += lossD_fake.item()
#     lossD_fake.backward()
    lossD = (lossD_real1+ lossD_real2+ lossD_fake)/3
    lossD.backward()

    lossD_item = lossD.item()
    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

#     label.fill_(real_label)
    output = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output, real_label)
#     lossA += lossA_real1.item()
#     lossA_real1.backward()

#     label.fill_(fake_label)
    output1 = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output1, fake_label)
#     lossA += lossA_real2.item()
#     lossA_real2.backward()

#     label.fill_(fake_label)
    output2 = neta(faked)
    lossA_fake = F.binary_cross_entropy(output2, fake_label)
#     lossA += lossA_fake.item()
#     lossA_fake.backward()
    lossA = (lossA_real1+ lossA_real2 +lossA_fake)/3
    lossA.backward()
    
    lossA_item = lossA.item()
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)
#     label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, real_label)
#     lossG += lossGD.item()
#     lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
#     label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, real_label)
#     lossG += lossGA.item()
#     lossGA.backward()
    lossG = (lossGD + lossGA)/2
    lossG.backward()
    
    lossG_item = lossG.item()
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 5000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
    if iteration % 5000==0 or iteration==10 or iteration==500:
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                           opts=dict(
                               title='GaitGAN',
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                     win=win,
                     update='append')


# WGAN from gaitgan

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.RMSprop(netg.parameters(), lr=lr/2)
optimD = optim.RMSprop(netd.parameters(), lr=lr/3)
optimA = optim.RMSprop(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, n_critic = {}, target={} \n'.format(
            epoches, lr, batchSize, n_critic, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
#         label.fill_(real_label)
#         lossD_real1 = F.binary_cross_entropy(output, label)
        lossD_real1 = -th.mean(output)
        lossD += lossD_real1.item()
        lossD_real1.backward()

#         label.fill_(real_label)
        output1 = netd(noass_label)
#         lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD_real2 = -th.mean(output1)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
#         label.fill_(fake_label)
        output2 = netd(fake)
#         lossD_fake = F.binary_cross_entropy(output2, label)
        lossD_fake = th.mean(output2)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()
        
        for p in netd.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake = netg(img).detach()
        faked = th.cat((img, fake), 1)

#         label.fill_(real_label)
        output1 = neta(assd)
#         lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA_real1 = -th.mean(output1)
        lossA += lossA_real1.item()
        lossA_real1.backward()

#         label.fill_(fake_label)
        output = neta(noassd)
#         lossA_real2 = F.binary_cross_entropy(output, label)
        lossA_real2 = th.mean(output)
        lossA += lossA_real2.item()
        lossA_real2.backward()

#         label.fill_(fake_label)
        output = neta(faked)
#         lossA_fake = F.binary_cross_entropy(output, label)
        lossA_fake = th.mean(output)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()
    
        for p in neta.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        # update G
        if i % n_critic == 0:
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
        
        


 # WGAN-GP

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 5
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.999

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=lr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=lr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=lr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, lr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        output = netd(ass_label)
#         label.fill_(real_label)
#         lossD_real1 = F.binary_cross_entropy(output, label)
        lossD_real1 = -th.mean(output)
        lossD_ += lossD_real1
        lossD += lossD_real1.item()
#         lossD_real1.backward()

#         label.fill_(real_label)
        output1 = netd(noass_label)
#         lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD_real2 = -th.mean(output1)
        lossD_ += lossD_real2
        lossD += lossD_real2.item()
#         lossD_real2.backward()

        fake = netg(img).detach()
#         label.fill_(fake_label)
        output2 = netd(fake)
#         lossD_fake = F.binary_cross_entropy(output2, label)
        lossD_fake = th.mean(output2)
        lossD_ += lossD_fake
        lossD += lossD_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
#         lossD_fake.backward()
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
        lossD_.backward()

        optimD.step()
        
#         for p in netd.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake = netg(img).detach()
        faked = th.cat((img, fake), 1)

#         label.fill_(real_label)
        output1 = neta(assd)
#         lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA_real1 = -th.mean(output1)
        lossA += lossA_real1.item()
        lossA_ += lossA_real1
#         lossA_real1.backward()

#         label.fill_(fake_label)
        output = neta(noassd)
#         lossA_real2 = F.binary_cross_entropy(output, label)
        lossA_real2 = th.mean(output)
        lossA += lossA_real2.item()
        lossA_ += lossA_real2
#         lossA_real2.backward()

#         label.fill_(fake_label)
        output = neta(faked)
#         lossA_fake = F.binary_cross_entropy(output, label)
        lossA_fake = th.mean(output)
        lossA += lossA_fake.item()
        lossA_ += lossA_fake
#         lossA_fake.backward()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
        
        lossA_.backward()
        optimA.step()
    
#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
        if i % n_critic == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossG_ += lossGD
#             lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossG_ += lossGA
#             lossGA.backward()
            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        
        


# Self attention Gait GAN

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
#         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
#         for p in netd.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
#         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()
    
#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 1.803581416606903, ErrA = 0.5441253483295441, ErrD = 0.0010343268513679504, Gattn=-0.004043725319206715, Dattn=0.0027265192475169897, Aattn=-0.0009774332866072655
Epoch = 4, ErrG = -0.10958673059940338, ErrA = 0.6583949668953816, ErrD = 0.629629373550415, Gattn=-0.0077371979132294655, Dattn=0.0034518165048211813, Aattn=-0.0013394292909651995
Epoch = 6, ErrG = 0.09220033884048462, ErrA = 0.6679098382592201, ErrD = 0.6703111926714579, Gattn=-0.010174904018640518, Dattn=0.0024460263084620237, Aattn=-0.0011066164588555694
Epoch = 8, ErrG = 0.1579294204711914, ErrA = 0.6587392246971527, ErrD = 0.6465316427250704, Gattn=-0.0114434864372015, Dattn=0.002010645344853401, Aattn=-0.0011061591794714332
Epoch = 10, ErrG = 0.10164174437522888, ErrA = 0.6548924278467894, ErrD = 0.6376114524900913, Gattn=-0.01221468858420848

Epoch = 94, ErrG = 1.0812082439661026, ErrA = 0.4288232872883479, ErrD = 0.1426642636458079, Gattn=-0.017339183017611504, Dattn=0.053903575986623764, Aattn=-0.08669409900903702
Epoch = 96, ErrG = 1.5783872604370117, ErrA = 0.2625628237922986, ErrD = 0.16173414389292398, Gattn=-0.017416546121239662, Dattn=0.054508160799741745, Aattn=-0.08799809962511063
Epoch = 98, ErrG = 2.169441819190979, ErrA = 0.4320705917974313, ErrD = 0.06250281135241191, Gattn=-0.017494086176156998, Dattn=0.055219389498233795, Aattn=-0.08943309634923935
Epoch = 100, ErrG = 1.6283391118049622, ErrA = 0.3502696355183919, ErrD = 0.0533318929374218, Gattn=-0.017577383667230606, Dattn=0.05584494397044182, Aattn=-0.09072363376617432
Epoch = 102, ErrG = 1.0961385071277618, ErrA = 0.3472533921400706, ErrD = 0.014911803727348646, Gattn=-0.017615215852856636, Dattn=0.05641499161720276, Aattn=-0.09240487217903137
Epoch = 104, ErrG = 1.8259952068328857, ErrA = 0.29744549840688705, ErrD = 0.0259802242120107, Gattn=-0.01768806

Epoch = 188, ErrG = 1.051015704870224, ErrA = 0.3440873647729556, ErrD = 0.09372586881120999, Gattn=-0.018866702914237976, Dattn=0.07592242956161499, Aattn=-0.152693971991539
Epoch = 190, ErrG = 1.3339873552322388, ErrA = 0.1746849243839582, ErrD = 0.0, Gattn=-0.018872074782848358, Dattn=0.07637832313776016, Aattn=-0.15404507517814636
Epoch = 192, ErrG = 0.6782128512859344, ErrA = 0.20041380698482195, ErrD = 0.1122073382139206, Gattn=-0.018854817375540733, Dattn=0.07646005600690842, Aattn=-0.15481069684028625
Epoch = 194, ErrG = 1.5105884075164795, ErrA = 0.2428871418039004, ErrD = 0.021860559781392414, Gattn=-0.018846334889531136, Dattn=0.07689046859741211, Aattn=-0.1562051773071289
Epoch = 196, ErrG = 1.8334134817123413, ErrA = 0.23865054051081339, ErrD = 0.017774879932403564, Gattn=-0.018830683082342148, Dattn=0.07689428329467773, Aattn=-0.15722136199474335
Epoch = 198, ErrG = 1.5439207553863525, ErrA = 0.2118163506189982, ErrD = 0.0, Gattn=-0.018809666857123375, Dattn=0.07762090861

Epoch = 284, ErrG = 1.936140537261963, ErrA = 0.14560741186141968, ErrD = 0.0, Gattn=-0.018368065357208252, Dattn=0.08641564100980759, Aattn=-0.19420760869979858
Epoch = 286, ErrG = 2.139888644218445, ErrA = 0.2522600603600343, ErrD = 0.04358568725486597, Gattn=-0.01837107725441456, Dattn=0.08665243536233902, Aattn=-0.1948973387479782
Epoch = 288, ErrG = 2.1219953298568726, ErrA = 0.19758380701144537, ErrD = 0.024789308508237202, Gattn=-0.018361741676926613, Dattn=0.08664605766534805, Aattn=-0.19593855738639832
Epoch = 290, ErrG = 1.462594985961914, ErrA = 0.13774899890025458, ErrD = 0.0, Gattn=-0.018352556973695755, Dattn=0.08703877776861191, Aattn=-0.19658644497394562
Epoch = 292, ErrG = 1.485458254814148, ErrA = 0.1793712923924128, ErrD = 0.019940711557865143, Gattn=-0.018329614773392677, Dattn=0.08683347702026367, Aattn=-0.19724419713020325
Epoch = 294, ErrG = 1.6909927129745483, ErrA = 0.148132536560297, ErrD = 0.0, Gattn=-0.01831866428256035, Dattn=0.08718614280223846, Aattn=-0.1

Epoch = 382, ErrG = 1.6105122566223145, ErrA = 0.09804915885130565, ErrD = 0.0, Gattn=-0.018001016229391098, Dattn=0.09606310725212097, Aattn=-0.22301067411899567
Epoch = 384, ErrG = 1.7628583312034607, ErrA = 0.07388554513454437, ErrD = 0.001740458110968272, Gattn=-0.017987677827477455, Dattn=0.09583239257335663, Aattn=-0.22347046434879303
Epoch = 386, ErrG = 1.4438501596450806, ErrA = 0.17930086764196554, ErrD = 0.0, Gattn=-0.017983175814151764, Dattn=0.09603842347860336, Aattn=-0.22390501201152802
Epoch = 388, ErrG = 1.5444098114967346, ErrA = 0.1995938258866469, ErrD = 0.0, Gattn=-0.017996136099100113, Dattn=0.09599293768405914, Aattn=-0.22441866993904114
Epoch = 390, ErrG = 1.4427493810653687, ErrA = 0.14457718407114348, ErrD = 0.0, Gattn=-0.017989136278629303, Dattn=0.09661310166120529, Aattn=-0.2250378280878067
Epoch = 392, ErrG = 2.050027906894684, ErrA = 0.25690524155894917, ErrD = 0.020779189964135487, Gattn=-0.017986100167036057, Dattn=0.0968441292643547, Aattn=-0.2254928350

Epoch = 482, ErrG = 1.081003338098526, ErrA = 0.18928182125091553, ErrD = 0.007905049249529839, Gattn=-0.017970630899071693, Dattn=0.10012809932231903, Aattn=-0.24540957808494568
Epoch = 484, ErrG = 1.8542426228523254, ErrA = 0.08972975114981334, ErrD = 0.0, Gattn=-0.017977707087993622, Dattn=0.10007581859827042, Aattn=-0.2457437366247177
Epoch = 486, ErrG = 1.701265037059784, ErrA = 0.10392284641663234, ErrD = 0.0, Gattn=-0.017973357811570168, Dattn=0.10037685930728912, Aattn=-0.24605385959148407
Epoch = 488, ErrG = 1.3160787224769592, ErrA = 0.12496997912724812, ErrD = 0.0, Gattn=-0.01797950454056263, Dattn=0.10074695199728012, Aattn=-0.24635791778564453
Epoch = 490, ErrG = 1.5796875357627869, ErrA = 0.11253867546717326, ErrD = 0.004873812198638916, Gattn=-0.017977053299546242, Dattn=0.10112224519252777, Aattn=-0.2466353476047516
Epoch = 492, ErrG = 1.4395246505737305, ErrA = 0.14348854621251425, ErrD = 0.0, Gattn=-0.017977261915802956, Dattn=0.10114476829767227, Aattn=-0.24718122184

Epoch = 580, ErrG = 1.2939506769180298, ErrA = 0.15656782935063043, ErrD = 0.0, Gattn=-0.01804419234395027, Dattn=0.10669845342636108, Aattn=-0.26195403933525085
Epoch = 582, ErrG = 1.975185513496399, ErrA = 0.06648607862492402, ErrD = 0.0, Gattn=-0.01803814247250557, Dattn=0.10671323537826538, Aattn=-0.2624295651912689
Epoch = 584, ErrG = 1.273537278175354, ErrA = 0.08141165723403294, ErrD = 0.021209922929604847, Gattn=-0.018050437793135643, Dattn=0.10698935389518738, Aattn=-0.26275166869163513
Epoch = 586, ErrG = 1.6718144416809082, ErrA = 0.0966839889685313, ErrD = 0.0, Gattn=-0.018065040931105614, Dattn=0.10744070261716843, Aattn=-0.26285025477409363
Epoch = 588, ErrG = 1.5926133394241333, ErrA = 0.11501884460449219, ErrD = 0.010489542037248611, Gattn=-0.018059520050883293, Dattn=0.10765602439641953, Aattn=-0.2629193663597107
Epoch = 590, ErrG = 1.2004563808441162, ErrA = 0.22287214236954847, ErrD = 0.019935589283704758, Gattn=-0.018073109909892082, Dattn=0.10807644575834274, Aattn

Epoch = 678, ErrG = 1.7263803482055664, ErrA = 0.048660725355148315, ErrD = 0.0, Gattn=-0.01818220689892769, Dattn=0.11455850303173065, Aattn=-0.2779906392097473
Epoch = 680, ErrG = 2.0219327807426453, ErrA = 0.13956951970855394, ErrD = 0.0, Gattn=-0.01818864606320858, Dattn=0.11488695442676544, Aattn=-0.2786276936531067
Epoch = 682, ErrG = 1.2504432797431946, ErrA = 0.2537495046854019, ErrD = 0.04140931119521459, Gattn=-0.01819939725100994, Dattn=0.1145668476819992, Aattn=-0.27890244126319885
Epoch = 684, ErrG = 1.566731572151184, ErrA = 0.09741202493508656, ErrD = 0.02307614249487718, Gattn=-0.01819445751607418, Dattn=0.11450346559286118, Aattn=-0.27960073947906494
Epoch = 686, ErrG = 1.5053852796554565, ErrA = 0.052854251116514206, ErrD = 0.0, Gattn=-0.018204430118203163, Dattn=0.11471088975667953, Aattn=-0.27985864877700806
Epoch = 688, ErrG = 1.4517470002174377, ErrA = 0.06959662213921547, ErrD = 0.0, Gattn=-0.018193934112787247, Dattn=0.11498140543699265, Aattn=-0.280081331729888

# combine siamese and SAGAN 

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 5

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())
        d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
#         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
#         for p in netd.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
#         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()
    
#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake, A= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        # constrain on generator
        fake_ass, P = netg(ass_label)
        fake_noass, N = netg(noass_label)
        lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
        lossG_ += lossTriplet
        lossG += lossTriplet.item()
#         lossTriplet.backward()
        
        lossG_ = lossG_/3
        lossG_.backward(retain_graph=True)
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

        